## Association Rule Mining - Pipeline Example

This notebook presents a basic pipeline to perform association rule analysis on a read-in dataset, based on the apriori algorithm. The main steps are elaborated below:

* [Imports](#imports)
* [Path and input settings](#pathinput)
* [Notes](#notes)
* [Functions](#functions)
* [Execution](#exe)

### Imports <a name="imports"></a>

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

### Path and input settings <a name="pathinput"></a>

In [2]:
my_path = "./"
my_file = "Training_courses_2019.xlsx"
sheet_name = "courses"

path_dict = {
    "lift": my_path + "Metric-lift/",
    "confidence ratio": my_path + "Metric-confidence-ratio/",
    "confidence difference": my_path + "Metric-confidence-diff/"
            }

In [3]:
df = pd.read_excel(my_path + my_file, sheet_name=sheet_name)

In [4]:
df.head()

,TICKET,Decision Trees,Building Predictive Models,Intro to CHAID,Classification and Clustering,Data Entry,FastTrack,Intermediate Techniques,Intro to Statistic platform,Intro to Platform & Statistics,Maps,Perceptual Mapping,Market Segmentation,Neural Networks,Scripting,Intro to Statistics,Tables,Time Series,ANOVA models
0,2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,16,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,32,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,33,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,36,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,1,0


In [5]:
# metric = "lift"        # controlling metric used for thresholding
# min_threshold = 1      # metric threshold

# min_rulesupport = 0    # minimum rule support

# min_support = 0.2      # minimum antecedent support
# min_confidence = 0.3   # minimum confidence

max_antecedents = 5    # maximum size of antecedent set
max_consequents = 1    # maximum size of consequent set

### Notes <a name="notes"></a>

**NOTE1**: The `pandas` documentation recommends using `DataFrame.to_numpy()` instead of `DataFrame.values`.

**NOTE2**: From the python documentation, `itertools.combinations(iterable, r)` returns `r` length subsequences of elements from the input `iterable`. Combinations are emitted in lexicographic sort order. So, if the input `iterable` is sorted, the combination tuples will be produced in sorted order. Elements are treated as unique based on their position, not on their value. So if the input elements are unique, there will be no repeat values in each combination.

**NOTE3**: The input to the `metric_dict` lambda functions in `association_rules()` is always a tuple with the following three support values:

* `support_tuple[0]`: support( *antecedent* &#8594; *consequent* )

* `support_tuple[1]`: support( *antecedent* )

* `support_tuple[2]`: support( *consequent* )

**NOTE4**: The purpose of `np.vectorize` is to transform functions which are not numpy-aware (e.g. take floats as input and return floats as output) into functions that can operate on (and return) numpy arrays.

**NOTE5**: A `frozenset` is immutable, and its items are not intrinsically ordered, which is a convenient feature for the keys of a dictionary (e.g. `frequent_items_dict`).

### Functions <a name="functions"></a>

In [6]:
def apriori(df, min_rulesupport=0, use_colnames=False,
            max_len=None, col_start=1):
    """
    Input arguments:
    * `df`
      pandas DataFrame with one-hot encoded columns embedding transactional information. The values in
      these columns are expected to be 1/0 or True/False. If there are columns with non-transactional
      information, these are assumed to be grouped in `df.columns[:col_start]`. See argument `col_start`
      for more details.
    * `min_rulesupport`
      Float representing the minimum threshold for the support value of the itemset under consideration
      to be considered frequent. By default (0), no minimum is imposed, which leaves `max_len` as the
      only way to narrow down the itemset search. In any case, filtering is enabled when resorting to
      metric thresholds upon calling the function `association_rules()`.
    * `use_colnames`
      Boolean indicating if the indices of the item columns pertaining to itemsets should be handled as
      integers (False, default) or with their string names (True).
    * `max_len`
      Integer imposing a maximum itemset size when searching for frequent itemsets. The search, if not
      interrupted by then due to non-compliance with `min_rulesupport`, will not continue for itemsets
      larger than this specified size. By default (None) no size restraint is imposed and hence only
      `min_rulesupport` could terminate the search before exhausting all possible itemsets.
    * `col_start`
      Index of the column from which all remaining columns are transactional, i.e. number of columns
      with non-transactional information. These columns are ignored throughout the process of searching
      for frequent itemsets. By default (1) it is assumed that only one such column exists in `df` (the very
      first one).
    
    Returns:
    * pandas DataFrame with two columns (['support', 'itemsets']) collecting the frequent itemsets compliant
      with the search conditions as well as their corresponding support values.
    """
    # See Note1
    X = df.to_numpy()
    column_indices = np.arange(col_start, X.shape[1])
    nrows, ncols = X.shape[0], X.shape[1]-col_start
    
    support = X[:,col_start:].sum(axis=0) / nrows
    
    support_dict = {1: support[support >= min_rulesupport]}
    itemset_dict = {1: column_indices[support >= min_rulesupport].reshape(-1, 1)}
    
    max_itemset = 1
    
    while max_itemset and max_itemset < (max_len or float('inf')):
        next_max_itemset = max_itemset + 1
        # See Note2
        combos = combinations(np.unique(itemset_dict[max_itemset].flatten()),
                              r=next_max_itemset)
        frequent_items = []
        frequent_items_support = []

        for c in combos:
            together = X[:, c].sum(axis=1) == len(c)
            support = together.sum() / nrows
            if support >= min_rulesupport:
                frequent_items.append(c)
                frequent_items_support.append(support)

        if frequent_items:
            itemset_dict[next_max_itemset] = np.array(frequent_items)
            support_dict[next_max_itemset] = np.array(frequent_items_support)
            max_itemset = next_max_itemset
        else:
            max_itemset = 0
        
    all_res = []
    
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([i for i in itemset_dict[k]])

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['support', 'itemsets']
    
    if use_colnames:
        mapping = {idx: item for idx, item in enumerate(df.columns)}
        res_df['itemsets'] = res_df['itemsets'].apply(lambda x: [mapping[i] for i in x])
        
    res_df = res_df.reset_index(drop=True)
    
    return res_df

In [7]:
def association_rules(df_freq,
                      metric="lift", min_threshold=1,
                      max_antecedents=None, max_consequents=None,
                      min_confidence=0.8, min_support=0.1):
    """
    Input arguments:
    * `df_freq`
      pandas DataFrame with two columns (['support', 'itemsets']) as per output from function `apriori()`.
    * `metric`
      String signifying the controlling metric by which to filter rules already compliant with support and
      confidence thresholds. The mined rules are sorted by this metric in decreasing order. By default,
      "lift" is assumed as the controlling metric. See `metric_dict` definition to inspect available metrics.
    * `min_threshold`
      Minimum value that `metric` needs to attain for a rule to be considered and returned. By default it
      is set to 1, associated to the default metric "lift".
    * `max_antecedents`
      Maximum size of the antecedent set considered for any rule. By default (None), no limitation is imposed
      on this size, which means that all possible antecedent sets within each itemset are exhausted when
      mining rules.
    * `max_consequents`
      Maximum size of the consequent set considered for any rule. By default (None), no limitation is imposed
      on this size, which means that all possible consequent sets within each itemset are exhausted when
      mining rules.
    * `min_confidence`
      Minimum value that the confidence of a rule needs to attain for it to be considered and returned.
      Defaults to 0.8.
    * `min_support`
      Minimum value that the support of the antecedent needs to attain for a rule to be considered and returned.
      Defaults to 0.1.
    
    Returns:
    * pandas DataFrame with all the mined association rules compliant with the thresholds (if no rules could be
      found for the given set of input arguments, a None object is returned instead). Each row represents a
      rule, and the columns are intuitively labelled as:
          ["antecedent(s)",
           "consequent(s)",
           "antecedent frequency",
           "antecedent support",
           "consequent support", 
           "confidence",
           "rule support",
           "lift",
           "confidence difference",
           "confidence ratio",
           "size antec.set",
           "size conseq.set",
           "rule ID"]
    """
    # See Note3
    metric_dict = {
        "antecedent support": lambda t: t[1],
        "consequent support": lambda t: t[2], #Prior
        "confidence": lambda t: t[0]/t[1],    #Posterior
        "rule support": lambda t: t[0],
        "confidence difference": lambda t: abs(t[0]/t[1] - t[2]),
        "confidence ratio": lambda t: 1 - np.minimum(t[0]/t[1], t[2])/np.maximum(t[0]/t[1], t[2]),
        "lift": lambda t: (t[0]/t[1])/t[2]
    }

    columns_ordered = [
        "antecedent support",
        "consequent support", 
        "confidence",
        "rule support",
        "lift",
        "confidence difference",
        "confidence ratio"
        ]

    # See Note1
    keys = df_freq['itemsets'].to_numpy()
    values = df_freq['support'].to_numpy()
    # See Note4 and Note5
    frozenset_vect = np.vectorize(lambda x: frozenset(x))
    frequent_items_dict = dict(zip(frozenset_vect(keys), values))
    
    rule_antecedents = []
    rule_consequents = []
    rule_supports = []

    n_transactions = len(df.index)

    for k in frequent_items_dict.keys():
        sAC = frequent_items_dict[k]
        # Adjust iteration boundaries to the restraints on antecedent/consequent set size
        if max_antecedents:
            sizeA = min(len(k)-1, max_antecedents)
        else:
            sizeA = len(k)-1
        if max_consequents:
            sizeC = max(len(k) - 1 - max_consequents, 0)
        else:
            sizeC = 0
        
        for idx in range(sizeA, sizeC, -1):
            for c in combinations(k, r=idx):
                antecedent = frozenset(c)
                consequent = k.difference(antecedent)

                sA = frequent_items_dict[antecedent]
                sC = frequent_items_dict[consequent]
                instances = sA * n_transactions

                support_tuple = (sAC, sA, sC)
                # Filtering off itemsets without support for antecendent and/or consequent
                if not sC or not sA: continue
                    
                if metric_dict['antecedent support'](support_tuple) >= min_support:
                    if metric_dict['confidence'](support_tuple) >= min_confidence:
                        if metric_dict[metric](support_tuple) >= min_threshold:
                            rule_antecedents.append(antecedent)
                            rule_consequents.append(consequent)
                            rule_supports.append([len(antecedent), len(consequent), instances, sAC, sA, sC])
                            
    if not rule_supports:
        print ("WARNING: No rules were found with the specified thresholds.")
        return None
    
    rule_supports = np.array(rule_supports).T.astype(float)
    numbantec = rule_supports[0]
    numbconseq = rule_supports[1]
    instances = rule_supports[2].astype(int)
    sAC = rule_supports[3]
    sA = rule_supports[4]
    sC = rule_supports[5]
    support_tuple = (sAC, sA, sC)

    dfrules = pd.DataFrame(data=list(zip(rule_antecedents, rule_consequents, instances)),
                           columns=["antecedent(s)", "consequent(s)", "antecedent frequency"])
    
    # Cast the frozensets to strings when using the colnames
    try:
        dfrules["antecedent(s)"] = dfrules["antecedent(s)"].apply(lambda x: ', '.join(list(x))).astype("unicode")
        dfrules["consequent(s)"] = dfrules["consequent(s)"].apply(lambda x: ', '.join(list(x))).astype("unicode")
    except:
        pass
        
    for m in columns_ordered:
        dfrules[m] = metric_dict[m](support_tuple)

    dfrules["size antec.set"] = numbantec.astype(int)
    dfrules["size conseq.set"] = numbconseq.astype(int)

    dfrules = dfrules.sort_values(metric, ascending=False)

    dfrules["rule ID"] = dfrules.index + 1
    number = ["s", ""][len(dfrules) == 1]
    print("INFO: {} rule".format(str(len(dfrules))) + number + " found.")
    
    return dfrules                

In [8]:
def rule_mining(df,
                min_rulesupport=0, use_colnames=False,
                max_len=None, col_start=1,
                metric="lift", min_threshold=1,
                max_antecedents=None, max_consequents=None,
                min_confidence=0.8, min_support=0.1):
    """
    Input arguments as per functions `apriori()` and `association_rules()`.
    Returns the pandas DataFrame resulting from `association_rules()`.
    
    This function acts as wrapper and interface between `apriori()` and `association_rules()`, making
    sure that the kwargs passed to these functions are consistent when changed from their default values,
    particularly those that default to None.
    """
    # Basic sanity checks amongst input arguments
    if max_len and max_antecedents and max_consequents:
        if (max_antecedents + max_consequents) != max_len:
            print("""ERROR: For consistency purposes between the size of the frequent itemsets retrieved
            and their split into antecedent and consequent with size restraints, `max_len` has to be equal
            to the sum of `max_antecedents` and `max_consequents` if all three arguments are specified.
            Adjust their values accordingly or leave `max_len` as `None`.""")
            return None
        
    if max_len is None and max_antecedents and max_consequents:
        print("""INFO: `max_len` will be set to {} to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.""".format(max_antecedents + max_consequents))
        max_len = max_antecedents + max_consequents
        
    if max_len and max_antecedents and not max_consequents:
        if not max_antecedents < max_len:
            print("""INFO: Although `max_antecedents` has been set to {0}, only antecedent sets
            of size up to {1} will be considered, consistently with given `max_len`. If this is not
            intended behaviour, adjust the value of the relevant argument.""".format(max_antecedents, max_len-1))
            
    if max_len and not max_antecedents and max_consequents:
        if not max_consequents < max_len:
            print("""INFO: Although `max_consequents` has been set to {0}, only consequent sets
            of size up to {1} will be considered, consistently with given `max_len`. If this is not
            intended behaviour, adjust the value of the relevant argument.""".format(max_consequents, max_len-1))
            
    if max_len and not max_antecedents and not max_consequents:
        if max_len > len(df.columns[col_start:]):
            print("""INFO: Although `max_len` has been set to {0}, only {1} columns of the given
            DataFrame are available and hence this constitutes the largest itemset size that will
            be considered. Check the value of `col_start`, otherwise set `max_len` to None or adjust
            its value appropriately.""".format(max_len, len(df.columns[col_start:])))
            
    if max_len is None and max_antecedents and not max_consequents:
        if max_antecedents > len(df.columns[col_start:]):
            print("""INFO: Although `max_antecedents` has been set to {0}, only {1} columns of the
            given DataFrame are available and hence this constitutes the largest itemset size that will
            be considered. Check the value of `col_start`, otherwise set `max_antecedents` to None or
            adjust its value appropriately.""".format(max_antecedents, len(df.columns[col_start:])))
            
    if max_len is None and not max_antecedents and max_consequents:
        if max_consequents > len(df.columns[col_start:]):
            print("""INFO: Although `max_consequents` has been set to {0}, only {1} columns of the
            given DataFrame are available and hence this constitutes the largest itemset size that will
            be considered. Check the value of `col_start`, otherwise set `max_consequents` to None or
            adjust its value appropriately.""".format(max_consequents, len(df.columns[col_start:])))
    
    # Call function to extract frequent itemsets
    support_itemsets = apriori(df, min_rulesupport, use_colnames, max_len, col_start)
    
    # Call function to mine rules 
    rules = association_rules(support_itemsets,
                              metric, min_threshold,
                              max_antecedents, max_consequents,
                              min_confidence, min_support)
    
    return rules

### Execution <a name="exe"></a>

In [9]:
metric = "lift"
lift_grid = [1.0]*6 + [0.5]*6
rulesupport_grid = [0]*12
support_grid = ([0.2]*3 + [0.1]*3) * 2
confidence_grid = [0.5, 0.4, 0.3]*4

for i, (l, rs, s, c) in enumerate(zip(lift_grid, rulesupport_grid, support_grid, confidence_grid)):
    tmp = rule_mining(df,
                      use_colnames=True, min_rulesupport=rs,
                      metric=metric, min_threshold=l,
                      max_antecedents=max_antecedents, max_consequents=max_consequents,
                      min_confidence=c, min_support=s)
    
    tmp_path = path_dict[metric] + metric + "_c" + str(i+1) + ".csv"
    tmp.to_csv(tmp_path, index=False)

INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 1 rule found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 4 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 9 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 6 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 17 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 31 rule

In [10]:
metric = "confidence ratio"
confratio_grid = [0.5]*3 + [0.7]*3 + [0.8]*3 + [0.9]*3
rulesupport_grid = [0.01]*12
support_grid = [0]*12
confidence_grid = [0.5, 0.4, 0.3]*4

for i, (cr, rs, s, c) in enumerate(zip(confratio_grid, rulesupport_grid, support_grid, confidence_grid)):
    tmp = rule_mining(df,
                      use_colnames=True, min_rulesupport=rs,
                      metric=metric, min_threshold=cr,
                      max_antecedents=max_antecedents, max_consequents=max_consequents,
                      min_confidence=c, min_support=s)
    
    tmp_path = path_dict[metric] + metric + "_c" + str(i+1) + ".csv"
    tmp.to_csv(tmp_path, index=False)

INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 870 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 958 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 1113 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 444 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
INFO: 482 rules found.
INFO: `max_len` will be set to 6 to avoid searching for itemsets beyond the given
        restraints on the number of antecedents and consequents.
IN